# Load Oxford Street data

In [2]:
# Install FiftyOne
!pip install fiftyone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import cv2
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import fiftyone as fo
import fiftyone.zoo as foz

from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Loading and count adding

In [59]:
data = pd.read_csv('/content/drive/My Drive/Oxford/TownCentre-groundtruth.top', names= ['personNumber', 'frameNumber',
                                'headNumber', 'bodyNumber', 'headLeft', 'headTop', 'headRight', 'headBottom', 'bodyLeft', 'bodyTop', 'bodyRight', 'bodyBottom'])

data = data.drop(['headNumber','personNumber', 'bodyNumber', 'headLeft', 'headTop', 'headRight', 'bodyLeft',
                  'bodyTop','bodyRight','bodyBottom','headBottom'], axis='columns')

count_data = data.drop_duplicates(subset='frameNumber', keep='first')

frame_counts = data['frameNumber'].value_counts(sort = False).reset_index()
frame_counts.columns = ['frameNumber', 'count']

count_data = count_data.merge(frame_counts, on='frameNumber')
count_data


3091


,frameNumber,count
0,0,14
1,1,14
2,2,14
3,3,14
4,4,14
...,...,...
3086,3086,8
3087,3087,8
3088,3088,8
3089,3089,8


In [19]:
# Paths to files - adjust output directory for Google drive etc
video_path = 'CNN-to-Evaluate-Social-Distancing-Measures/02_data/Oxford_Data/TownCentreXVID.mp4'
output_dir = '/content/drive/My Drive/Oxford/images'

In [ ]:
# Open the video file
'''cap = cv2.VideoCapture(video_path)

frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Define the name and path for each frame
    frame_filename = os.path.join(output_dir, f'{frame_number}.jpg')

    # Save the frame as a JPG file
    cv2.imwrite(frame_filename, frame)

    frame_number += 1


cap.release()
cv2.destroyAllWindows()'''


## Data preprocessing

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()  # Convert the image to tensor
])

# Function to load an image and convert to tensor
def load_image_as_tensor(image_path):
    # Open image
    image = Image.open(image_path).convert('RGB')
    # Apply the transform to the image
    tensor_image = transform(image)
    return tensor_image

# Function to process a range of images
def process_images_in_range(image_folder, start_frame, end_frame):
    tensors = {}
    for frame in range(start_frame, end_frame + 1):
        image_path = os.path.join(image_folder, f'{frame}.jpg')  # Assuming images are named as frame numbers
        if os.path.exists(image_path):
            # Load the image and convert it to a tensor
            image_tensor = load_image_as_tensor(image_path)
            tensors[frame] = image_tensor
        else:
            print(f"Image for frame {frame} not found.")
    return tensors

# Function to display the tensor as an image
def show_image(tensor):
    # Convert tensor to numpy array and transpose to [H, W, C]
    np_image = tensor.permute(1, 2, 0).numpy()
    plt.imshow(np_image)
    plt.axis('off')
    plt.show()

# Path to the folder containing images

start_frame = 1  # Starting frame number
end_frame = 3090   # Ending frame number

# Process images in the specified range
image_tensors = process_images_in_range(output_dir, start_frame, end_frame)



## Data Loader